In [558]:
import numpy as np
import pandas as pd

pd.options.display.max_rows = 20

pd.options.display.max_colwidth = 80

pd.options.display.max_columns = 20


np.random.seed(12345)



import matplotlib.pyplot as plt

plt.rc("figure", figsize=(10, 6)) # Configura matplotlib para que todas
# las figuras tengan un tamaño predeterminado de 10 pulgadas de ancho 
# por 6 pulgadas de alto. rc es una función de matplotlib que se 
# utiliza para configurar parámetros globales.

np.set_printoptions(precision=4, suppress=True)
# Configura numpy para mostrar los números con una 
# precisión de 4 decimales y suprimir el uso de 
# notación científica (suppress=True) en la salida impresa.

# 3. Tratamiento de datos (Wrangling): Unir (Join) ,combinar (Combine) y remodelar (Reshape)

 ## 3.1 Indexación jerárquica

Pandas permite tener múltiples (dos o más) niveles de índice en un eje. 

In [559]:
data = pd.Series(np.random.uniform(size=9),
        index=[["Madrid", "Madrid", "Madrid", "Segovia", "Segovia", "Asturias", "Asturias", "Compostela", "Compostela"],
        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

#Multiíndice (`MultiIndex`)

Madrid      1    0.929616
            2    0.316376
            3    0.183919
Segovia     1    0.204560
            3    0.567725
Asturias    1    0.595545
            2    0.964515
Compostela  2    0.653177
            3    0.748907
dtype: float64

In [560]:
data.index

MultiIndex([(    'Madrid', 1),
            (    'Madrid', 2),
            (    'Madrid', 3),
            (   'Segovia', 1),
            (   'Segovia', 3),
            (  'Asturias', 1),
            (  'Asturias', 2),
            ('Compostela', 2),
            ('Compostela', 3)],
           )

In [561]:
data["Madrid"]

1    0.929616
2    0.316376
3    0.183919
dtype: float64

In [562]:
data.loc[["Asturias", "Compostela"]]

Asturias    1    0.595545
            2    0.964515
Compostela  2    0.653177
            3    0.748907
dtype: float64

In [563]:
data.loc[:, 2] # Selecciona todas las filas (:) y selecciona la columna (2)

Madrid        0.316376
Asturias      0.964515
Compostela    0.653177
dtype: float64

### **Ejemplo 3.1**

In [564]:
data_1 = {
    'Store': ['ITa', 'ITa', 'ITa', 'ITb', 'ITb', 'ITb'],
    'Product': ['A', 'A', 'B', 'A', 'B', 'B'],
    'Date': ['2024-07-01', '2024-07-02', '2024-07-01', '2024-07-01', '2024-07-02', '2024-07-03'],
    'Sales': [10, 150, 200, 300, 400, 50]
}
df = pd.DataFrame(data_1)

df['Date'] = pd.to_datetime(df['Date'])  # Convertir la columna 'Date' a tipo datetime
df

,Store,Product,Date,Sales
0,ITa,A,2024-07-01,10
1,ITa,A,2024-07-02,150
2,ITa,B,2024-07-01,200
3,ITb,A,2024-07-01,300
4,ITb,B,2024-07-02,400
5,ITb,B,2024-07-03,50


**Configurar a Multiindex**

In [565]:
df.set_index(['Store', 'Product', 'Date'], inplace=True) #Tiene 3 multi-índice
df

Sales
Store Product Date             
ITa   A       2024-07-01     10
              2024-07-02    150
      B       2024-07-01    200
ITb   A       2024-07-01    300
      B       2024-07-02    400
              2024-07-03     50

In [566]:
data

Madrid      1    0.929616
            2    0.316376
            3    0.183919
Segovia     1    0.204560
            3    0.567725
Asturias    1    0.595545
            2    0.964515
Compostela  2    0.653177
            3    0.748907
dtype: float64

In [567]:
data.unstack()

,1,2,3
Asturias,0.595545,0.964515,NaN
Compostela,NaN,0.653177,0.748907
Madrid,0.929616,0.316376,0.183919
Segovia,0.204560,NaN,0.567725


La operación inversa de desapilar (`unstack`) es apilar (`stack`):

In [568]:
data.unstack().stack()

Asturias    1    0.595545
            2    0.964515
Compostela  2    0.653177
            3    0.748907
Madrid      1    0.929616
            2    0.316376
            3    0.183919
Segovia     1    0.204560
            3    0.567725
dtype: float64

### Vamos a desapilar el dataframe del ejemplo 3.1

In [569]:
df

Sales
Store Product Date             
ITa   A       2024-07-01     10
              2024-07-02    150
      B       2024-07-01    200
ITb   A       2024-07-01    300
      B       2024-07-02    400
              2024-07-03     50

In [570]:
df.unstack()

Sales                      
Date          2024-07-01 2024-07-02 2024-07-03
Store Product                                 
ITa   A             10.0      150.0        NaN
      B            200.0        NaN        NaN
ITb   A            300.0        NaN        NaN
      B              NaN      400.0       50.0

In [571]:
df.unstack('Store')

Sales       
Store                 ITa    ITb
Product Date                    
A       2024-07-01   10.0  300.0
        2024-07-02  150.0    NaN
B       2024-07-01  200.0    NaN
        2024-07-02    NaN  400.0
        2024-07-03    NaN   50.0

In [572]:
df.unstack('Product')

Sales       
Product               A      B
Store Date                    
ITa   2024-07-01   10.0  200.0
      2024-07-02  150.0    NaN
ITb   2024-07-01  300.0    NaN
      2024-07-02    NaN  400.0
      2024-07-03    NaN   50.0

#### Con un DataFrame, cualquiera de los ejes puede tener un índice jerárquico:

In [721]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["Conservadores", "Conservadores", "Democratas", "Democratas"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                     ["Green", "Red", "Green"]])
frame

Ohio     Colorado
                Green Red    Green
Conservadores 1     0   1        2
              2     3   4        5
Democratas    1     6   7        8
              2     9  10       11

In [722]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

state               Ohio     Colorado
color              Green Red    Green
key1          key2                   
Conservadores 1        0   1        2
              2        3   4        5
Democratas    1        6   7        8
              2        9  10       11

In [723]:
frame.index.nlevels

2

In [724]:
frame.columns.nlevels

2

In [725]:
frame["Ohio"]

color               Green  Red
key1          key2            
Conservadores 1         0    1
              2         3    4
Democratas    1         6    7
              2         9   10

In [726]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"],
                          ["Green", "Red", "Green"]],
                          names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

## Reordenación y clasificación de niveles

In [727]:
frame

state               Ohio     Colorado
color              Green Red    Green
key1          key2                   
Conservadores 1        0   1        2
              2        3   4        5
Democratas    1        6   7        8
              2        9  10       11

In [728]:
frame.swaplevel("key1", "key2")

state               Ohio     Colorado
color              Green Red    Green
key2 key1                            
1    Conservadores     0   1        2
2    Conservadores     3   4        5
1    Democratas        6   7        8
2    Democratas        9  10       11

In [729]:
frame.sort_index()

state               Ohio     Colorado
color              Green Red    Green
key1          key2                   
Conservadores 1        0   1        2
              2        3   4        5
Democratas    1        6   7        8
              2        9  10       11

In [730]:
frame.sort_index(level=0)

state               Ohio     Colorado
color              Green Red    Green
key1          key2                   
Conservadores 1        0   1        2
              2        3   4        5
Democratas    1        6   7        8
              2        9  10       11

In [731]:
frame.sort_index(level=1)

state               Ohio     Colorado
color              Green Red    Green
key1          key2                   
Conservadores 1        0   1        2
Democratas    1        6   7        8
Conservadores 2        3   4        5
Democratas    2        9  10       11

In [732]:
frame.swaplevel(0, 1)

state               Ohio     Colorado
color              Green Red    Green
key2 key1                            
1    Conservadores     0   1        2
2    Conservadores     3   4        5
1    Democratas        6   7        8
2    Democratas        9  10       11

In [733]:
frame.swaplevel(0, 1).sort_index(level=0)
#

state               Ohio     Colorado
color              Green Red    Green
key2 key1                            
1    Conservadores     0   1        2
     Democratas        6   7        8
2    Conservadores     3   4        5
     Democratas        9  10       11

## Resumen estadístico por niveles

In [735]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [736]:
frame.groupby(level="color", axis="columns").sum()

C:\Users\CursosTardes\AppData\Local\Temp\ipykernel_14640\775557097.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level="color", axis="columns").sum()


color               Green  Red
key1          key2            
Conservadores 1         2    1
              2         8    4
Democratas    1        14    7
              2        20   10

## Indexación con las columnas de un DataFrame

In [740]:
frame = pd.DataFrame({"Francia": range(7), "Italia": range(7, 0, -1),
                      "Malta": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "Portugal": [0, 1, 2, 0, 1, 2, 3]})
frame

,Francia,Italia,Malta,Portugal
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [742]:
frame2 = frame.set_index(["Malta", "Portugal"])
frame2

Francia  Italia
Malta Portugal                 
one   0               0       7
      1               1       6
      2               2       5
two   0               3       4
      1               4       3
      2               5       2
      3               6       1

In [743]:
frame.set_index(["Malta", "Portugal"], drop=False)

Francia  Italia Malta  Portugal
Malta Portugal                                 
one   0               0       7   one         0
      1               1       6   one         1
      2               2       5   one         2
two   0               3       4   two         0
      1               4       3   two         1
      2               5       2   two         2
      3               6       1   two         3

In [744]:
frame2.reset_index()

,Malta,Portugal,Francia,Italia
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 3.2 Combinar y fusionar conjuntos de datos

## Uniones de DataFrames al estilo de las bases de datos

In [746]:
df1 = pd.DataFrame({"key": ["UIO", "UIO", "GYE", "CUE", "GYE", "GYE", "UIO"],
                    "data1": pd.Series(range(7), dtype="Int64")})
df1

,key,data1
0,UIO,0
1,UIO,1
2,GYE,2
3,CUE,3
4,GYE,4
5,GYE,5
6,UIO,6


In [747]:
df2 = pd.DataFrame({"key": ["GYE", "UIO", "CUE"],
                    "data2": pd.Series(range(3), dtype="Int64")})

df2

,key,data2
0,GYE,0
1,UIO,1
2,CUE,2


In [752]:
print('El merge entre el df1 y df2: ')
display(pd.merge(df1, df2))
print('\nEl merge entre df2 y df1: ')
pd.merge(df2, df1)

El merge entre el df1 y df2: 


,key,data1,data2
0,UIO,0,1
1,UIO,1,1
2,GYE,2,0
3,CUE,3,2
4,GYE,4,0
5,GYE,5,0
6,UIO,6,1



El merge entre df2 y df1: 


,key,data2,data1
0,GYE,0,2
1,GYE,0,4
2,GYE,0,5
3,UIO,1,0
4,UIO,1,1
5,UIO,1,6
6,CUE,2,3


In [753]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,UIO,0,1
1,UIO,1,1
2,GYE,2,0
3,CUE,3,2
4,GYE,4,0
5,GYE,5,0
6,UIO,6,1


In [759]:
df3 = pd.DataFrame({"lkey": ["Mamiferos", "Mamiferos", "Reptiles", "Herviboros", "Mamiferos", "Mamiferos", "Reptiles"],
                    "data1": pd.Series(range(7), dtype="Int64")})

df3

,lkey,data1
0,Mamiferos,0
1,Mamiferos,1
2,Reptiles,2
3,Herviboros,3
4,Mamiferos,4
5,Mamiferos,5
6,Reptiles,6


In [760]:
df4 = pd.DataFrame({"rkey": ["Mamiferos", "Reptiles", "Aves"],
                    "data2": pd.Series(range(3), dtype="Int64")})
df4

,rkey,data2
0,Mamiferos,0
1,Reptiles,1
2,Aves,2


In [761]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,Mamiferos,0,Mamiferos,0
1,Mamiferos,1,Mamiferos,0
2,Reptiles,2,Reptiles,1
3,Mamiferos,4,Mamiferos,0
4,Mamiferos,5,Mamiferos,0
5,Reptiles,6,Reptiles,1


In [762]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,CUE,3,2
1,GYE,2,0
2,GYE,4,0
3,GYE,5,0
4,UIO,0,1
5,UIO,1,1
6,UIO,6,1


In [763]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer") 

,lkey,data1,rkey,data2
0,NaN,<NA>,Aves,2
1,Herviboros,3,NaN,<NA>
2,Mamiferos,0,Mamiferos,0
3,Mamiferos,1,Mamiferos,0
4,Mamiferos,4,Mamiferos,0
5,Mamiferos,5,Mamiferos,0
6,Reptiles,2,Reptiles,1
7,Reptiles,6,Reptiles,1


In [764]:
df1 = pd.DataFrame({"key": ["Sofia", "Sofia", "Ana", "Carlos", "Sofia", "Ben"],
                    "data1": pd.Series(range(6), dtype="Int64")})

df1

,key,data1
0,Sofia,0
1,Sofia,1
2,Ana,2
3,Carlos,3
4,Sofia,4
5,Ben,5


In [765]:
df2 = pd.DataFrame({"key": ["Sofia", "Ben", "Ana", "Ben", "Dari"],
                    "data2": pd.Series(range(5), dtype="Int64")})
df2

,key,data2
0,Sofia,0
1,Ben,1
2,Ana,2
3,Ben,3
4,Dari,4


In [766]:
pd.merge(df1, df2, on="key", how="left") 

,key,data1,data2
0,Sofia,0,0
1,Sofia,1,0
2,Ana,2,2
3,Carlos,3,<NA>
4,Sofia,4,0
5,Ben,5,1
6,Ben,5,3


In [767]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,Sofia,0,0
1,Sofia,1,0
2,Ana,2,2
3,Ben,5,1
4,Sofia,4,0
5,Ben,5,3


------------------------------
# Ejercicio 9
--------------------------------------

**Partiendo de los siguientes datos:**

In [768]:
# Datos del departamento de Ingeniería
data_ingenieria = {
    'Empleado': ['Alice', 'Bob', 'Charlie', 'Dan', 'Ellen'],
    'Salario': [80000, 85000, 90000, 87000, 88000],
    'Departamento': ['Ingeniería'] * 5
}
# Datos del departamento de Marketing
data_marketing = {
    'Empleado': ['David', 'Eve', 'Frank', 'Gina', 'Hank'],
    'Salario': [70000, 75000, 80000, 72000, 78000],
    'Departamento': ['Marketing'] * 5
}
# Datos del departamento de Ventas
data_ventas = {
    'Empleado': ['Grace', 'Heidi', 'Ivan', 'Jack', 'Karen'],
    'Salario': [65000, 70000, 75000, 68000, 69000],
    'Departamento': ['Ventas'] * 5
}
# Datos del departamento de Recursos Humanos
data_rrhh = {
    'Empleado': ['Judy', 'Ken', 'Laura', 'Mike', 'Nina'],
    'Salario': [60000, 65000, 70000, 64000, 66000],
    'Departamento': ['Recursos Humanos'] * 5
}

#### *Pregunta 9.1 Crear los dataframes de cada departamento y luego concatenarlos para obtener un único dataframe final*

In [782]:
ingenieria = pd.DataFrame(data_ingenieria)
marketing = pd.DataFrame(data_marketing)
ventas = pd.DataFrame(data_ventas)
rrhh = pd.DataFrame(data_rrhh)

In [787]:
df = pd.concat([ingenieria, marketing, ventas, rrhh])
df.head(4)

,Empleado,Salario,Departamento
0,Alice,80000,Ingeniería
1,Bob,85000,Ingeniería
2,Charlie,90000,Ingeniería
3,Dan,87000,Ingeniería


----------------------
# Ejercicio 10 
--------------------------

Aplicación de `combine_first` para completar datos de prueba con datos de entrenamiento:

**Datos**

In [788]:
# Datos de entrenamiento
train_data = {
    'Producto': ['A', 'B', 'C', 'D'],
    'Ventas': [100, 150, 200, 250]
}

# Datos de prueba con valores faltantes
test_data = {
    'Producto': ['A', 'B', 'C', 'E'],
    'Ventas': [np.nan, 160, np.nan, 180]
}

**Pregunta 10.1: Crear los dataframes**  

In [794]:
train = pd.DataFrame(train_data)
test = pd.DataFrame(test_data)
display('El dataframe de train: \n', train)
display('El dataframe de test: \n', test)

'El dataframe de train: \n'

,Producto,Ventas
0,A,100
1,B,150
2,C,200
3,D,250


'El dataframe de test: \n'

,Producto,Ventas
0,A,NaN
1,B,160.0
2,C,NaN
3,E,180.0


**Pregunta 10.2: Combine los datos y guarde los resultados como df_test_combined**

In [805]:
df_test_combined = train.combine_first(test)
df_test_combined

,Producto,Ventas
0,A,100.0
1,B,150.0
2,C,200.0
3,D,250.0


-------------------------------------
# Ejercicio 11: Actualización de Datos
------------------------

Supongamos que tenemos datos antiguos y nuevos sobre el inventario de productos y queremos actualizar los datos antiguos con la nueva información.

**Datos:**

In [797]:
old_inventory = {
    'Producto': ['Laptop', 'Smartphone', 'Tablet'],
    'Cantidad': [10, 20, 15],
    'Precio': [1000, 800, 300]
}

# Datos nuevos del inventario con actualizaciones
new_inventory = {
    'Producto': ['Laptop', 'Smartphone', 'Tablet'],
    'Cantidad': [12, 18, 17],
    'Precio': [950, 850, 310]
}

**Pregunta 11.1: Crear los dataframes**

In [806]:
old = pd.DataFrame(old_inventory)
new = pd.DataFrame(new_inventory)

display('El dataframe old: \n', old)
display('El dataframe new: \n', new)


'El dataframe old: \n'

,Producto,Cantidad,Precio
0,Laptop,10,1000
1,Smartphone,20,800
2,Tablet,15,300


'El dataframe new: \n'

,Producto,Cantidad,Precio
0,Laptop,12,950
1,Smartphone,18,850
2,Tablet,17,310


**Pregunta 11.2: Actualizar usando combine_first**

In [804]:
inventario = old.combine_first(new)
inventario

,Producto,Cantidad,Precio
0,Laptop,10,1000
1,Smartphone,20,800
2,Tablet,15,300


## 3.3 Reshaping con `stack` y `unstack()`

Existen varias operaciones básicas para reorganizar datos tabulares. Se denominan operaciones de `reshape` o `pivot`.

### Remodelación (Reshaping) con indexación jerárquica

La indexación jerárquica proporciona una forma coherente de reorganizar los datos en un DataFrame. Existen dos acciones principales:

`stack`: Esto "gira" o pivota de las columnas de los datos a las filas.

`unstack`: Pivota de las filas a las columnas.

Se ilustrarán estas operaciones con una serie de ejemplos. Consideremos un pequeño DataFrame con arrays de cadenas como índices de fila y columna:

In [680]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["Uno", "Dos", "Tres"],
                    name="numero"))
data

numero,Uno,Dos,Tres
state,,,
Ohio,0,1,2
Colorado,3,4,5


Utilizando el método `stack` en estos datos, las columnas pivotan en las filas, produciendo una Serie:

In [681]:
result = data.stack()
result

state     numero
Ohio      Uno       0
          Dos       1
          Tres      2
Colorado  Uno       3
          Dos       4
          Tres      5
dtype: int32

A partir de una Serie indexada jerárquicamente, puede reorganizar los datos de nuevo en un DataFrame con `unstack()`:

In [682]:
result.unstack()

numero,Uno,Dos,Tres
state,,,
Ohio,0,1,2
Colorado,3,4,5


Por defecto, el nivel más interno está desapilado (igual que la pila). Puedes desapilar(unstack) un nivel diferente pasando un número o nombre de nivel:

In [683]:
result.unstack(level=0)

state,Ohio,Colorado
numero,,
Uno,0,3
Dos,1,4
Tres,2,5


In [684]:
result.unstack(level="state")

state,Ohio,Colorado
numero,,
Uno,0,3
Dos,1,4
Tres,2,5


El desapilamiento puede introducir datos que faltan si no se encuentran todos los valores del nivel en cada subgrupo:

In [685]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s1

a    0
b    1
c    2
d    3
dtype: Int64

In [686]:
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
s2

c    4
d    5
e    6
dtype: Int64

In [687]:
data2 = pd.concat([s1, s2], keys=["one", "two"])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

Si deseara filtrar por defecto los datos que faltan podemos usar `unstack`, por lo que la operación es más fácilmente invertible:

In [688]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


si quiero volver a la forma inicial:

In [689]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [690]:
data2.unstack().stack(dropna=False)
# convierte las columnas de nuevo en un índice jerárquico, manteniendo los NaN.

C:\Users\CursosTardes\AppData\Local\Temp\ipykernel_14640\3853392162.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data2.unstack().stack(dropna=False)


one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

Cuando se desapila (unstack) en un DataFrame, el nivel desapilado se convierte en el nivel más bajo del resultado:

In [691]:
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))
df

side             left  right
state    numero             
Ohio     Uno        0      5
         Dos        1      6
         Tres       2      7
Colorado Uno        3      8
         Dos        4      9
         Tres       5     10

In [692]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
numero                             
Uno       0        3     5        8
Dos       1        4     6        9
Tres      2        5     7       10

Al igual que con `unstack`, al llamar a `stack` podemos indicar el nombre del eje a apilar:

df.unstack(level="state").stack(level="side")

#### Ejercicio 12. Uso de stack y unstack para hacer 'reshaping' sobre dataframes:

**Datos**

In [693]:
data = {
    'Región': ['Norte', 'Norte', 'Norte', 'Sur', 'Sur', 'Sur', 'Este', 'Este', 'Este', 'Oeste', 'Oeste', 'Oeste'],
    'Producto': ['Laptop', 'Smartphone', 'Tablet', 'Laptop', 'Smartphone', 'Tablet', 'Laptop', 'Smartphone', 'Tablet', 'Laptop', 'Smartphone', 'Tablet'],
    'Q1': [150, 200, 50, 100, 80, 30, 120, 60, 90, 110, 130, 40],
    'Q2': [170, 210, 60, 110, 85, 35, 125, 65, 95, 115, 135, 45],
    'Q3': [160, 220, 55, 105, 82, 32, 122, 62, 92, 112, 132, 42],
    'Q4': [180, 230, 65, 115, 90, 40, 130, 70, 100, 120, 140, 50]
}

**Pregunta 12.1: A partir de `data` crear un dataframe multi-indice (en fila) que tenga esta forma:**

| Región | Producto   | Q1  | Q2  | Q3  | Q4  |
|--------|------------|-----|-----|-----|-----|
| Norte  | Laptop     | 150 | 170 | 160 | 180 |
|        | Smartphone | 200 | 210 | 220 | 230 |
|        | Tablet     |  50 |  60 |  55 |  65 |
| Sur    | Laptop     | 100 | 110 | 105 | 115 |
|        | Smartphone |  80 |  85 |  82 |  90 |
|        | Tablet     |  30 |  35 |  32 |  40 |
| Este   | Laptop     | 120 | 125 | 122 | 130 |
|        | Smartphone |  60 |  65 |  62 |  70 |
|        | Tablet     |  90 |  95 |  92 | 100 |
| Oeste  | Laptop     | 110 | 115 | 112 | 120 |
|        | Smartphone | 130 | 135 | 132 | 140 |
|        | Tablet     |  40 |  45 |  42 |  50 |


**Donde: `Región` y `Producto` tienen una jerarquía de índice de nivel 2**

**Pregunta 12.2: Usar `unstack` para transformar el índice de nivel inferior (Producto) en columnas.**

**Pregunta 12.3: Revertir el `unstack` anterior usando `stack`**

#### Ejercicio 13. Datos de transacciones bancarias

**Datos:**

In [694]:
data = {
    'Sucursal': ['Norte', 'Norte', 'Norte', 'Norte', 'Sur', 'Sur', 'Sur', 'Sur', 'Este', 'Este', 'Este', 'Este', 'Oeste', 'Oeste', 'Oeste', 'Oeste'],
    'Cuenta': ['Corriente', 'Ahorros', 'Crédito', 'Inversiones', 'Corriente', 'Ahorros', 'Crédito', 'Inversiones', 'Corriente', 'Ahorros', 'Crédito', 'Inversiones', 'Corriente', 'Ahorros', 'Crédito', 'Inversiones'],
    'Año': [2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021],
    'Q1': [50000, 150000, 30000, 70000, 45000, 120000, 25000, 65000, 52000, 130000, 27000, 72000, 49000, 140000, 28000, 68000],
    'Q2': [52000, 155000, 31000, 71000, 46000, 125000, 26000, 66000, 53000, 135000, 28000, 73000, 50000, 145000, 29000, 69000],
    'Q3': [51000, 160000, 32000, 72000, 47000, 130000, 27000, 67000, 54000, 140000, 29000, 74000, 51000, 150000, 30000, 70000],
    'Q4': [53000, 165000, 33000, 73000, 48000, 135000, 28000, 68000, 55000, 145000, 30000, 75000, 52000, 155000, 31000, 71000]
}

**Pregunta 13.1: Crear un dataframe con índices jerarquicos (en filas) de nivel 3**

**Pregunta 13.2: Usar `unstack` para transformar el índice de nivel inferior ('Año') en columnas**

**Pregunta 13.3: Revertir el dataframe anterior a su forma original, usar `stack`**

### Pasar del formato "largo" (Long) al "ancho (Wide)

Una forma habitual de almacenar múltiples series temporales (series de fechas) en bases de datos y archivos CSV es lo que a veces se denomina formato largo o apilado (`stacked format`). En este formato, los valores individuales se representan mediante una única fila en una tabla, en lugar de múltiples valores por fila.

Carguemos algunos datos de ejemplo y hagamos una pequeña limpieza de series temporales y otros datos:

In [695]:
data = pd.read_csv("macrodata.csv")
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


Esta dataset contiene una variedad de indicadores económicos clave que se utilizan para analizar el estado de la economía, incluyendo medidas de producción, consumo, inversión, gasto gubernamental, oferta monetaria, tasas de interés, desempleo, inflación y población. Estos datos son cruciales para economistas y analistas que estudian el rendimiento económico y realizan pronósticos económicos.

**Columnas:**  

- `year`: El año en el que se registraron los datos.  

- `quarter`: El trimestre del año en el que se registraron los datos. Los trimestres están numerados del 1 al 4.
- `realgdp`: Producto Interno Bruto (PIB) real. Es una medida del valor de todos los bienes y servicios producidos en una economía, ajustada por inflación.
- `realcons`: Consumo real. Representa el gasto total de los hogares en bienes y servicios, ajustado por inflación.
- `realinv`: Inversión real. Incluye la inversión en bienes de capital, como maquinaria y edificios, ajustada por inflación.
- `realgovt`: Gasto gubernamental real. El gasto total del gobierno en bienes y servicios, ajustado por inflación.
- `realdpi`: Ingreso personal disponible real. Es el ingreso total que tienen los individuos después de impuestos, ajustado por inflación.
- `cpi`: Índice de Precios al Consumidor (CPI). Una medida que examina el promedio ponderado de los precios de una canasta de bienes y servicios de consumo, y se utiliza para medir la inflación.
- `m1`: Oferta monetaria M1. Incluye el efectivo en circulación y los depósitos a la vista en bancos, una medida de la oferta de dinero en la economía.
- `tbilrate`: Tasa de interés de los Bonos del Tesoro a 3 meses. La tasa de rendimiento de los bonos del Tesoro de Estados Unidos con vencimiento a 3 meses.
- `unemp`: Tasa de desempleo. El porcentaje de la población activa que está desempleada y buscando empleo.
- `pop`: Población. La población total del país en millones.
- `infl`: Tasa de inflación. La tasa a la que suben los precios de los bienes y servicios, normalmente medida como un cambio porcentual anual en el CPI.
- `realint`: Tasa de interés real. La tasa de interés nominal ajustada por la inflación.

In [696]:
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]

In [697]:
data.head()

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2


En primer lugar, se va utilizar `pandas.PeriodIndex` para representar intervalos de tiempo en lugar de puntos en el tiempo, lo veremos con más detalle en el tema de Series temporales. La finalidad combinar las columnas de `year` y `quarter` y establecer el índice para que consista en valores `datetime` al final de cada trimestre:

In [698]:
periods = pd.PeriodIndex(year=data.pop("year"),
                         quarter=data.pop("quarter"),
                         name="date")
periods

C:\Users\CursosTardes\AppData\Local\Temp\ipykernel_14640\1359989538.py:1: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  periods = pd.PeriodIndex(year=data.pop("year"),


PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

- `data.pop("year")` y `data.pop("quarter")`:

    - `data.pop("year")`: Esta función elimina la columna "year" del DataFrame data y devuelve los valores de esa columna.
    - `data.pop("quarter")`: Similar a pop("year"), esta función elimina la columna "quarter" del DataFrame data y devuelve los valores de esa columna.
    - Al usar `pop()`, las columnas "year" y "quarter" se eliminan del DataFrame original data, y sus valores se utilizan para crear el `PeriodIndex`.

In [699]:
data.index = periods.to_timestamp("D")

El método `to_timestamp()` convierte un `PeriodIndex` a un `DatetimeIndex`.  

- `"D"` es un argumento opcional que especifica la frecuencia del DatetimeIndex resultante. `"D"` significa que queremos que la frecuencia sea diaria.
En el caso de trimestres, `to_timestamp("D")` toma la fecha de inicio del trimestre como el timestamp.

In [700]:
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


Aquí se ha utilizado el método `pop` en el DataFrame, que devuelve una columna al mismo tiempo que la elimina del DataFrame.

A continuación, se selecciona un subconjunto de columnas y se le da el nombre `"item"` al índice de columnas:

In [701]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])
data.columns.name = "item"
data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


Por último, se ha hecho un reshape con `stack`, convierte los nuevos niveles de índice en columnas con `reset_index` y, por último, se le da el nombre `"value"` a la columna que contiene los valores de los datos:

In [702]:
long_data = (data.stack()
             .reset_index()
             .rename(columns={0: "value"}))

- `data.stack()`: Apila las columnas en una sola columna, transformando el DataFrame a un formato largo.  
- `reset_index()`: Restablece el índice, convirtiendo los índices apilados en columnas.  
- `rename(columns={0: "value"})`: Renombra la columna resultante de los valores apilados a `"value"`.

Ahora, `ldata` se ve así:

In [703]:
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


En este formato denominado largo para series temporales múltiples, cada fila de la tabla representa una única observación.

Los datos se almacenan con frecuencia de esta forma en bases de datos relacionales SQL, ya que un esquema fijo (nombres de columna y tipos de datos) permite que el número de valores distintos en la columna de `ítem` cambie a medida que se añaden datos a la tabla. 

En el ejemplo anterior, `date` y `item` suelen ser las claves primarias (en el lenguaje de las bases de datos relacionales), lo que ofrece integridad relacional y facilita las uniones (joins). En algunos casos, puede ser más difícil trabajar con los datos en este formato; es posible que prefiera tener un DataFrame que contenga una columna por cada valor de elemento distinto indexado por marcas de tiempo (timestamps) en la columna de fecha(date). El método `pivot` de DataFrame realiza exactamente esta transformación:

In [704]:
pivoted = long_data.pivot(index="date", columns="item",
                          values="value")

`long_data.pivot()`:  
        - El método `.pivot()` reorganiza el DataFrame, utilizando valores únicos de una columna como columnas nuevas y otra columna como índices.  
        - `index="date"`: Especifica que la columna "date" del DataFrame `long_data` se usará como índice del DataFrame resultante.  
        - `columns="item"`: Especifica que los valores únicos de la columna "item" (que contiene "realgdp", "infl", "unemp") se convertirán en nuevas columnas en el DataFrame resultante.  
        - `values="value"`: Especifica que los valores en la columna "value" del DataFrame long_data se usarán como valores en el DataFrame resultante.

In [705]:
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


Los dos primeros valores pasados son las columnas que se utilizarán, respectivamente, como índice de fila y de columna, y finalmente una columna de valor opcional para rellenar el DataFrame. Supongamos que tiene dos columnas de valores que desea remodelar (reshape) simultáneamente:

In [706]:
long_data["value2"] = np.random.standard_normal(len(long_data))
long_data[:10]

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,0.802926
1,1959-01-01,infl,0.000,0.575721
2,1959-01-01,unemp,5.800,1.381918
3,1959-04-01,realgdp,2778.801,0.000992
4,1959-04-01,infl,2.340,-0.143492
5,1959-04-01,unemp,5.100,-0.206282
6,1959-07-01,realgdp,2775.488,-0.222392
7,1959-07-01,infl,2.740,-1.682403
8,1959-07-01,unemp,5.300,1.811659
9,1959-10-01,realgdp,2785.204,-0.351305


In [707]:
long_data.index.name = None
# Establece el nombre del índice a None.
# Esto elimina cualquier nombre asignado
# previamente al índice del DataFrame long_data.

Omitiendo el último argumento, se obtiene un DataFrame con columnas jerárquicas:

In [708]:
pivoted = long_data.pivot(index="date", columns="item")

`pivot(index="date", columns="item")`: Reorganiza el DataFrame de modo que:  
    - `index="date"`: La columna "date" se convierte en el índice.  
    - `columns="item"`: Los valores únicos de la columna "item" se convierten en las nuevas columnas del DataFrame.

In [709]:
pivoted.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.575721  0.802926  1.381918
1959-04-01  2.34  2778.801   5.1 -0.143492  0.000992 -0.206282
1959-07-01  2.74  2775.488   5.3 -1.682403 -0.222392  1.811659
1959-10-01  0.27  2785.204   5.6  0.128317 -0.351305 -1.313554
1960-01-01  2.31  2847.699   5.2 -0.615939  0.498327  0.174072

In [710]:
pivoted["value"].head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


Tenga en cuenta que pivotar es equivalente a crear un índice jerárquico utilizando `set_index` seguido de una llamada a `unstack`:

In [711]:
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.575721  0.802926  1.381918
1959-04-01  2.34  2778.801   5.1 -0.143492  0.000992 -0.206282
1959-07-01  2.74  2775.488   5.3 -1.682403 -0.222392  1.811659
1959-10-01  0.27  2785.204   5.6  0.128317 -0.351305 -1.313554
1960-01-01  2.31  2847.699   5.2 -0.615939  0.498327  0.174072

#### **Ejercicio 14: Datos de ventas mensuales**

In [712]:
data = {
    'Tienda': ['Tienda_A', 'Tienda_A', 'Tienda_A', 'Tienda_A', 'Tienda_A', 'Tienda_A',
               'Tienda_B', 'Tienda_B', 'Tienda_B', 'Tienda_B', 'Tienda_B', 'Tienda_B',
               'Tienda_C', 'Tienda_C', 'Tienda_C', 'Tienda_C', 'Tienda_C', 'Tienda_C',
               'Tienda_D', 'Tienda_D', 'Tienda_D', 'Tienda_D', 'Tienda_D', 'Tienda_D',
               'Tienda_E', 'Tienda_E', 'Tienda_E', 'Tienda_E', 'Tienda_E', 'Tienda_E'],
    'Producto': ['Manzana', 'Manzana', 'Manzana', 'Pera', 'Pera', 'Pera',
                 'Manzana', 'Manzana', 'Manzana', 'Pera', 'Pera', 'Pera',
                 'Lechuga', 'Lechuga', 'Lechuga', 'Tomate', 'Tomate', 'Tomate',
                 'Lechuga', 'Lechuga', 'Lechuga', 'Tomate', 'Tomate', 'Tomate',
                 'Frijoles', 'Frijoles', 'Frijoles', 'Zanahoria', 'Zanahoria', 'Zanahoria'],
    'Año': [2020, 2020, 2020, 2020, 2020, 2020,
            2021, 2021, 2021, 2021, 2021, 2021,
            2020, 2020, 2020, 2020, 2020, 2020,
            2021, 2021, 2021, 2021, 2021, 2021,
            2020, 2020, 2020, 2020, 2020, 2020],
    'Mes': ['Enero', 'Febrero', 'Marzo', 'Enero', 'Febrero', 'Marzo',
            'Enero', 'Febrero', 'Marzo', 'Enero', 'Febrero', 'Marzo',
            'Enero', 'Febrero', 'Marzo', 'Enero', 'Febrero', 'Marzo',
            'Enero', 'Febrero', 'Marzo', 'Enero', 'Febrero', 'Marzo',
            'Enero', 'Febrero', 'Marzo', 'Enero', 'Febrero', 'Marzo'],
    'Ventas': [200, 220, 230, 150, 170, 180,
               180, 210, 220, 160, 190, 200,
               190, 200, 210, 170, 180, 190,
               210, 220, 230, 200, 210, 220,
               200, 210, 220, 210, 220, 230]
}

**Pregunta 14.1: Crear el dataframe con la información de arriba**

**Pregunta 14.2: Pivotar el DataFrame para que los meses sean columnas.**

**Pregunta 14.3: Usar `unstack` para convertir el nivel Producto en columnas.**

**Pregunta 14.4:  Usar `stack` para revertir la transformación y volver al formato anterior.**

**Pregunta 14.5: Crear un PeriodIndex y Convertirlo a Timestamps**
-  Se debe crear un índice de períodos basado en Año y Mes, y luego lo convertimos a timestamps.

### Pasar (Pivoting) del formato "ancho" (Wide) al "largo" (Long)

Una operación inversa al pivote para DataFrames es `pandas.melt`. En lugar de transformar una columna en muchas en un nuevo DataFrame, fusiona(merges) múltiples columnas en una, produciendo un DataFrame más largo que el de entrada. Veamos un ejemplo:

In [713]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1, 2, 3],
                   "B": [4, 5, 6],
                   "C": [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


La columna "key" puede ser un indicador de grupo, y las otras columnas son valores de datos. Al utilizar `pandas.melt`, debemos indicar qué columnas (si las hay) son indicadores de grupo. Usemos aquí `"key"` como único indicador de grupo:

In [714]:
melted = pd.melt(df, id_vars="key")
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


Usando `pivot`, podemos volver (reshape) al diseño original:

In [715]:
reshaped = melted.pivot(index="key", columns="variable",
                        values="value")
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


Dado que el resultado de `pivot` crea un índice a partir de la columna utilizada como etiquetas de fila, es posible que deseemos utilizar `reset_index` para volver a mover los datos a una columna:

In [716]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


También puede especificar un subconjunto de columnas para utilizarlas como columnas de valores:

In [717]:
pd.melt(df, id_vars="key", value_vars=["A", "B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


`pandas.melt` también se puede utilizar sin ningún identificador de grupo:

In [718]:
pd.melt(df, value_vars=["A", "B", "C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [719]:
pd.melt(df, value_vars=["key", "A", "B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


**Ejercicio 15: Datos de ventas. Uso de melt**

In [720]:
data = {
    'Tienda': ['Tienda_A', 'Tienda_A', 'Tienda_A', 'Tienda_A', 'Tienda_A', 'Tienda_A',
               'Tienda_B', 'Tienda_B', 'Tienda_B', 'Tienda_B', 'Tienda_B', 'Tienda_B',
               'Tienda_C', 'Tienda_C', 'Tienda_C', 'Tienda_C', 'Tienda_C', 'Tienda_C',
               'Tienda_D', 'Tienda_D', 'Tienda_D', 'Tienda_D', 'Tienda_D', 'Tienda_D',
               'Tienda_E', 'Tienda_E', 'Tienda_E', 'Tienda_E', 'Tienda_E', 'Tienda_E'],
    'Producto': ['Ropa', 'Ropa', 'Ropa', 'Calzado', 'Calzado', 'Calzado',
                 'Ropa', 'Ropa', 'Ropa', 'Calzado', 'Calzado', 'Calzado',
                 'Lencería', 'Lencería', 'Lencería', 'Calzado', 'Calzado', 'Calzado',
                 'Lencería', 'Lencería', 'Lencería', 'Calzado', 'Calzado', 'Calzado',
                 'Ropa', 'Ropa', 'Ropa', 'Lencería', 'Lencería', 'Lencería'],
    'Año': [2020, 2020, 2020, 2020, 2020, 2020,
            2021, 2021, 2021, 2021, 2021, 2021,
            2020, 2020, 2020, 2020, 2020, 2020,
            2021, 2021, 2021, 2021, 2021, 2021,
            2020, 2020, 2020, 2020, 2020, 2020],
    'Mes': ['Enero', 'Febrero', 'Marzo', 'Enero', 'Febrero', 'Marzo',
            'Enero', 'Febrero', 'Marzo', 'Enero', 'Febrero', 'Marzo',
            'Enero', 'Febrero', 'Marzo', 'Enero', 'Febrero', 'Marzo',
            'Enero', 'Febrero', 'Marzo', 'Enero', 'Febrero', 'Marzo',
            'Enero', 'Febrero', 'Marzo', 'Enero', 'Febrero', 'Marzo'],
    'Ventas': [200, 220, 230, 150, 170, 180,
               180, 210, 220, 160, 190, 200,
               190, 200, 210, 170, 180, 190,
               210, 220, 230, 200, 210, 220,
               200, 210, 220, 210, 220, 230],
    'Hora': ['10:00', '11:00', '12:00', '13:00', '14:00', '15:00',
             '16:00', '17:00', '18:00', '19:00', '20:00', '21:00',
             '09:00', '10:00', '11:00', '12:00', '13:00', '14:00',
             '15:00', '16:00', '17:00', '18:00', '19:00', '20:00',
             '10:00', '11:00', '12:00', '13:00', '14:00', '15:00']
}

**Pregunta 15.1: Crear el Dataframe**

**Pregunta 15.2: Crear un índice jerárquico utilizando las columnas Tienda, Producto y Año. Luego pivotamos el DataFrame para que los meses sean columnas.**

**Pregunta 15.3: Usar `stack` para convertir las columnas de meses en un índice y luego `unstack` para convertir el nivel `Producto` en columnas.**

**Pregunta 15.4: Reindexar el DataFrame para asegurarnos de que todas las combinaciones posibles de tiendas, productos y años estén presentes, rellenando con valores NaN donde no haya datos.**

**Pregunta 15.5: Utilizar `melt` para transformar el DataFrame de vuelta a un formato largo.**

**Pregunta 15.6: Crear un índice de períodos basado en Año, Mes y Hora, luego convertir a timestamps y ,finalmente, renombrar las columnas.**